In [1]:
import pandas as pd
import json
from datetime import datetime
import re
import os 

In [6]:
with open("./data/labeled_reviews.jsonl", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data)

In [7]:
df.head()

,제품명,브랜드,원가,판매가,리뷰수,평균별점,5점비율,4점비율,3점비율,2점비율,...,작성자,작성일자,별점,사용현황,세정력,자극도,거품,향,리뷰,annotation
0,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,용자입니다,2025.05.19,5점만점에 5점,,아주 만족해요,자극없이 순해요,거품이 풍성해요,아주 만족해요,일주일째 쓰는 중인데 좋은 거 같기도 하고 아닌 것 같기도 하고 한 달 뒤 후기로..ㄹ,"[[없음, [None, 0, 0], None]]"
1,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,레몽몽,2025.04.28,5점만점에 4점,,보통이에요,자극없이 순해요,거품이 풍성해요,아주 만족해요,거품이나 향은 무난한데 새치 커버력이 홍보 대비 좀 아쉽네요,"[[세정, [None, 0, 0], negative], [거품, [None, 0, ..."
2,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,k202401158****,2025.05.16,5점만점에 5점,재구매,아주 만족해요,자극없이 순해요,거품이 풍성해요,아주 만족해요,얼마 전에 구매했는데 쿠폰이 있어서 또 구매했어요 미리 쟁여 둘려 구요,"[[없음, [None, 0, 0], None]]"
3,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,k202401158****,2025.05.10,5점만점에 5점,,아주 만족해요,자극없이 순해요,거품이 풍성해요,아주 만족해요,염색 하지 않고 몇 년째 계속 사용하고 있어요 남편 덕분에 알게 되어서 지금까지 같...,"[[없음, [None, 0, 0], None]]"
4,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,sweetvio****,2025.05.06,5점만점에 5점,한달이상사용,보통이에요,보통이에요,보통이에요,보통이에요,남편 40대 되고 흰 머리가 많아 이걸 사줬는데 그레이 정도는 되는 것 같아요,"[[없음, [None, 0, 0], None]]"


In [8]:
k = pd.read_csv("./data/url_image.csv")
k.head()

,상품번호,상품명,이미지URL,리뷰,링크
0,A000000212961,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),https://image.oliveyoung.co.kr/cfimages/cf-goo...,https://www.oliveyoung.co.kr/store/goods/getGo...,NaN
1,A000000204247,라보에이치 두피쿨링&노세범샴푸 탈모증상완화 750+100ML 한정기획 / 단품,https://image.oliveyoung.co.kr/cfimages/cf-goo...,https://www.oliveyoung.co.kr/store/goods/getGo...,NaN
2,A000000183059,[6월 올영픽/신규향 듀오 출시] 어노브 딥 데미지 리페어 샴푸 500g 2입 2종 택1,https://image.oliveyoung.co.kr/cfimages/cf-goo...,https://www.oliveyoung.co.kr/store/goods/getGo...,NaN
3,A000000217893,[카이스트특허기술] 그래비티 헤어리프팅 샴푸 엑스트라 스트롱 475ml (단품/증정기획),https://image.oliveyoung.co.kr/cfimages/cf-goo...,https://www.oliveyoung.co.kr/store/goods/getGo...,NaN
4,A000000164470,롱테이크 샴푸 300+100ML 기획 외 단품 2종 택 1,https://image.oliveyoung.co.kr/cfimages/cf-goo...,https://www.oliveyoung.co.kr/store/goods/getGo...,NaN


In [9]:
k.rename(columns = {"이미지URL":"image", "리뷰":"url"}, inplace = True)

In [10]:
k = k.drop(columns = "링크")

In [11]:
k.head()

,상품번호,상품명,image,url
0,A000000212961,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),https://image.oliveyoung.co.kr/cfimages/cf-goo...,https://www.oliveyoung.co.kr/store/goods/getGo...
1,A000000204247,라보에이치 두피쿨링&노세범샴푸 탈모증상완화 750+100ML 한정기획 / 단품,https://image.oliveyoung.co.kr/cfimages/cf-goo...,https://www.oliveyoung.co.kr/store/goods/getGo...
2,A000000183059,[6월 올영픽/신규향 듀오 출시] 어노브 딥 데미지 리페어 샴푸 500g 2입 2종 택1,https://image.oliveyoung.co.kr/cfimages/cf-goo...,https://www.oliveyoung.co.kr/store/goods/getGo...
3,A000000217893,[카이스트특허기술] 그래비티 헤어리프팅 샴푸 엑스트라 스트롱 475ml (단품/증정기획),https://image.oliveyoung.co.kr/cfimages/cf-goo...,https://www.oliveyoung.co.kr/store/goods/getGo...
4,A000000164470,롱테이크 샴푸 300+100ML 기획 외 단품 2종 택 1,https://image.oliveyoung.co.kr/cfimages/cf-goo...,https://www.oliveyoung.co.kr/store/goods/getGo...


In [12]:
df = pd.merge(df, k, how = "left", left_on = "제품명", right_on = "상품명")

In [13]:
df = df.drop(columns = "상품명")

In [14]:
df.columns

Index(['제품명', '브랜드', '원가', '판매가', '리뷰수', '평균별점', '5점비율', '4점비율', '3점비율',
       '2점비율', '1점비율', '세정력/아주만족해요', '세정력/보통이에요', '세정력/다소아쉬워요', '자극도/자극없이순해요',
       '자극도/보통이에요', '자극도/자극이느껴져요', '거품/거품이풍성해요', '거품/보통이에요', '거품/거품이적어요',
       '향/아주만족해요', '향/보통이에요', '향/다소아쉬워요', '작성자', '작성일자', '별점', '사용현황', '세정력',
       '자극도', '거품', '향', '리뷰', 'annotation', '상품번호', 'image', 'url'],
      dtype='object')

In [15]:
df.head()

,제품명,브랜드,원가,판매가,리뷰수,평균별점,5점비율,4점비율,3점비율,2점비율,...,사용현황,세정력,자극도,거품,향,리뷰,annotation,상품번호,image,url
0,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,,아주 만족해요,자극없이 순해요,거품이 풍성해요,아주 만족해요,일주일째 쓰는 중인데 좋은 거 같기도 하고 아닌 것 같기도 하고 한 달 뒤 후기로..ㄹ,"[[없음, [None, 0, 0], None]]",A000000212961,https://image.oliveyoung.co.kr/cfimages/cf-goo...,https://www.oliveyoung.co.kr/store/goods/getGo...
1,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,,보통이에요,자극없이 순해요,거품이 풍성해요,아주 만족해요,거품이나 향은 무난한데 새치 커버력이 홍보 대비 좀 아쉽네요,"[[세정, [None, 0, 0], negative], [거품, [None, 0, ...",A000000212961,https://image.oliveyoung.co.kr/cfimages/cf-goo...,https://www.oliveyoung.co.kr/store/goods/getGo...
2,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,재구매,아주 만족해요,자극없이 순해요,거품이 풍성해요,아주 만족해요,얼마 전에 구매했는데 쿠폰이 있어서 또 구매했어요 미리 쟁여 둘려 구요,"[[없음, [None, 0, 0], None]]",A000000212961,https://image.oliveyoung.co.kr/cfimages/cf-goo...,https://www.oliveyoung.co.kr/store/goods/getGo...
3,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,,아주 만족해요,자극없이 순해요,거품이 풍성해요,아주 만족해요,염색 하지 않고 몇 년째 계속 사용하고 있어요 남편 덕분에 알게 되어서 지금까지 같...,"[[없음, [None, 0, 0], None]]",A000000212961,https://image.oliveyoung.co.kr/cfimages/cf-goo...,https://www.oliveyoung.co.kr/store/goods/getGo...
4,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,한달이상사용,보통이에요,보통이에요,보통이에요,보통이에요,남편 40대 되고 흰 머리가 많아 이걸 사줬는데 그레이 정도는 되는 것 같아요,"[[없음, [None, 0, 0], None]]",A000000212961,https://image.oliveyoung.co.kr/cfimages/cf-goo...,https://www.oliveyoung.co.kr/store/goods/getGo...


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181908 entries, 0 to 181907
Data columns (total 36 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   제품명          181908 non-null  object
 1   브랜드          181908 non-null  object
 2   원가           181908 non-null  object
 3   판매가          181908 non-null  object
 4   리뷰수          181908 non-null  object
 5   평균별점         181908 non-null  object
 6   5점비율         181908 non-null  object
 7   4점비율         181908 non-null  object
 8   3점비율         181908 non-null  object
 9   2점비율         181908 non-null  object
 10  1점비율         181908 non-null  object
 11  세정력/아주만족해요   181908 non-null  object
 12  세정력/보통이에요    181908 non-null  object
 13  세정력/다소아쉬워요   181908 non-null  object
 14  자극도/자극없이순해요  181908 non-null  object
 15  자극도/보통이에요    181908 non-null  object
 16  자극도/자극이느껴져요  181908 non-null  object
 17  거품/거품이풍성해요   181908 non-null  object
 18  거품/보통이에요     181908 non-null  object
 19  거품

In [17]:
df.shape

(181908, 36)

In [18]:
df.to_csv('./db_all.csv', index=False)

--- 
# utils

In [19]:
def clean_price(price_str):
    if pd.isna(price_str) or not price_str:
        return None
    return int(
        price_str.replace("\n", "").replace("원", "").replace(",", "").replace("~", "").strip()
    )

def percent_to_int(p):
    if pd.isna(p):
        return None
    return int(p.replace("%", "").strip())

def str_to_float(s):
    try:
        return float(s)
    except (ValueError, TypeError):
        return None

def str_to_int(s):
    try:
        return int(str(s).replace(",", "").strip())
    except (ValueError, TypeError):
        return None

def parse_date(date_str):
    try:
        return datetime.strptime(date_str.strip(), "%Y.%m.%d") 
    except (ValueError, TypeError):
        return None

def extract_score(s):
    try:
        matches = re.findall(r'(\d+)점', str(s))
        return int(matches[-1]) if matches else None
    except Exception:
        return None

def extract_labels(annotations):
    pair_lst = []
    aspect_lst = []
    for annotation in annotations:
        aspect = annotation[0]
        polarity = annotation[2]
        if aspect not in aspect_lst:
            aspect_lst.append(aspect)
            pair_lst.append((aspect, polarity))
    return pair_lst

entity_name = ['세정', '자극', '거품', '향', '가격', '머릿결', '탈모', '쿨링']
polarity_name = ['positive', 'negative', 'neutral']
pair_labels = [(entity, polarity) for polarity in polarity_name for entity in entity_name]
pair_labels.append(('없음', None))

def count_pairs(gp):
    count_dict = {pair: 0 for pair in pair_labels}
    for aspect, sentiment in gp[["aspect", "sentiment"]].values:
        if (aspect, sentiment) in count_dict:
            count_dict[(aspect, sentiment)] += 1
    return count_dict

def compute_ratio(gp_df):
    total = gp_df.groupby("aspect")["pair_count"].sum()
    gp_df["pair_ratio"] = gp_df.apply(
        lambda x: round(x["pair_count"] / total[x["aspect"]], 4) if total[x["aspect"]] != 0 else 0,
        axis=1
    )
    return gp_df

--- 
# df1 - Shampoo DB

In [20]:
df.columns

Index(['제품명', '브랜드', '원가', '판매가', '리뷰수', '평균별점', '5점비율', '4점비율', '3점비율',
       '2점비율', '1점비율', '세정력/아주만족해요', '세정력/보통이에요', '세정력/다소아쉬워요', '자극도/자극없이순해요',
       '자극도/보통이에요', '자극도/자극이느껴져요', '거품/거품이풍성해요', '거품/보통이에요', '거품/거품이적어요',
       '향/아주만족해요', '향/보통이에요', '향/다소아쉬워요', '작성자', '작성일자', '별점', '사용현황', '세정력',
       '자극도', '거품', '향', '리뷰', 'annotation', '상품번호', 'image', 'url'],
      dtype='object')

In [21]:
df1 = pd.DataFrame()

In [22]:
df1["shampoo"] = df["제품명"]
df1["brand"] = df["브랜드"]
df1["ori_price"] = df["원가"].apply(clean_price)
df1["sale_price"] = df["판매가"].apply(clean_price)
df1["review_count"] = df["리뷰수"].apply(str_to_int)
df1["avg_star"] = df["평균별점"].apply(str_to_float)
df1["star_5"] = df["5점비율"].apply(percent_to_int)
df1["star_4"] = df["4점비율"].apply(percent_to_int)
df1["star_3"] = df["3점비율"].apply(percent_to_int)
df1["star_2"] = df["2점비율"].apply(percent_to_int)
df1["star_1"] = df["1점비율"].apply(percent_to_int)
df1["c_3"] = df["세정력/아주만족해요"].apply(percent_to_int)
df1["c_2"] = df["세정력/보통이에요"].apply(percent_to_int)
df1["c_1"] = df["세정력/다소아쉬워요"].apply(percent_to_int)
df1["i_3"] = df["자극도/자극없이순해요"].apply(percent_to_int)
df1["i_2"] = df["자극도/보통이에요"].apply(percent_to_int)
df1["i_1"] = df["자극도/자극이느껴져요"].apply(percent_to_int)
df1["b_3"] = df["거품/거품이풍성해요"].apply(percent_to_int)
df1["b_2"] = df["거품/보통이에요"].apply(percent_to_int)
df1["b_1"] = df["거품/거품이적어요"].apply(percent_to_int)
df1["s_3"] = df["향/아주만족해요"].apply(percent_to_int)
df1["s_2"] = df["향/보통이에요"].apply(percent_to_int)
df1["s_1"] = df["향/다소아쉬워요"].apply(percent_to_int)
df1["url"] = df["url"] 
df1["image"] = df["image"]

In [23]:
df1.head()

,shampoo,brand,ori_price,sale_price,review_count,avg_star,star_5,star_4,star_3,star_2,...,i_2,i_1,b_3,b_2,b_1,s_3,s_2,s_1,url,image
0,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,78000.0,48280,145,4.6,78,13,6,1,...,22,4,68,30,2,60,36,4,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/cfimages/cf-goo...
1,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,78000.0,48280,145,4.6,78,13,6,1,...,22,4,68,30,2,60,36,4,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/cfimages/cf-goo...
2,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,78000.0,48280,145,4.6,78,13,6,1,...,22,4,68,30,2,60,36,4,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/cfimages/cf-goo...
3,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,78000.0,48280,145,4.6,78,13,6,1,...,22,4,68,30,2,60,36,4,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/cfimages/cf-goo...
4,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,78000.0,48280,145,4.6,78,13,6,1,...,22,4,68,30,2,60,36,4,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/cfimages/cf-goo...


In [24]:
df1 = df1.drop_duplicates()

In [25]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 475 entries, 0 to 180916
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   shampoo       475 non-null    object 
 1   brand         475 non-null    object 
 2   ori_price     287 non-null    float64
 3   sale_price    475 non-null    int64  
 4   review_count  475 non-null    int64  
 5   avg_star      475 non-null    float64
 6   star_5        475 non-null    int64  
 7   star_4        475 non-null    int64  
 8   star_3        475 non-null    int64  
 9   star_2        475 non-null    int64  
 10  star_1        475 non-null    int64  
 11  c_3           475 non-null    int64  
 12  c_2           475 non-null    int64  
 13  c_1           475 non-null    int64  
 14  i_3           475 non-null    int64  
 15  i_2           475 non-null    int64  
 16  i_1           475 non-null    int64  
 17  b_3           475 non-null    int64  
 18  b_2           475 non-null    in

In [26]:
df1.shape

(475, 25)

In [162]:
df1.to_csv('./db_shampoo.csv', index=False)

--- 
# df2 - Review DB

In [27]:
df2 = pd.DataFrame()

In [28]:
df2["shampoo"] = df["제품명"]
df2["writer"] = df["작성자"]
df2["review"] = df["리뷰"]
df2["created_at"] = df["작성일자"].apply(parse_date)
df2["star"] = df["별점"].apply(extract_score).astype("Int64")

In [29]:
df2.head()

,shampoo,writer,review,created_at,star
0,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),용자입니다,일주일째 쓰는 중인데 좋은 거 같기도 하고 아닌 것 같기도 하고 한 달 뒤 후기로..ㄹ,2025-05-19,5
1,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),레몽몽,거품이나 향은 무난한데 새치 커버력이 홍보 대비 좀 아쉽네요,2025-04-28,4
2,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),k202401158****,얼마 전에 구매했는데 쿠폰이 있어서 또 구매했어요 미리 쟁여 둘려 구요,2025-05-16,5
3,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),k202401158****,염색 하지 않고 몇 년째 계속 사용하고 있어요 남편 덕분에 알게 되어서 지금까지 같...,2025-05-10,5
4,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),sweetvio****,남편 40대 되고 흰 머리가 많아 이걸 사줬는데 그레이 정도는 되는 것 같아요,2025-05-06,5


In [30]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181908 entries, 0 to 181907
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   shampoo     181908 non-null  object        
 1   writer      181908 non-null  object        
 2   review      181908 non-null  object        
 3   created_at  181908 non-null  datetime64[ns]
 4   star        181908 non-null  Int64         
dtypes: Int64(1), datetime64[ns](1), object(3)
memory usage: 7.1+ MB


In [31]:
df2.shape

(181908, 5)

In [33]:
df2 = df2.drop_duplicates(subset = ["shampoo", "review", "writer"])

In [34]:
df2.shape

(178960, 5)

In [175]:
df2.to_csv('./db_review.csv', index=False)

--- 
# df3 - AspectSentiment 

In [35]:
df["labels"] = df["annotation"].map(extract_labels)

In [36]:
df3_data = []
for i in range(len(df)):
    review = df.loc[i, "리뷰"]
    shampoo = df.loc[i, "제품명"]
    for aspect, sentiment in df.loc[i, "labels"]:
        df3_data.append({
            "shampoo" : shampoo,
            "review" : review,
            "aspect" : aspect,
            "sentiment" : sentiment
        })
df3 = pd.DataFrame(df3_data)

In [37]:
df3.head()

,shampoo,review,aspect,sentiment
0,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),일주일째 쓰는 중인데 좋은 거 같기도 하고 아닌 것 같기도 하고 한 달 뒤 후기로..ㄹ,없음,None
1,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),거품이나 향은 무난한데 새치 커버력이 홍보 대비 좀 아쉽네요,세정,negative
2,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),거품이나 향은 무난한데 새치 커버력이 홍보 대비 좀 아쉽네요,거품,neutral
3,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),거품이나 향은 무난한데 새치 커버력이 홍보 대비 좀 아쉽네요,향,neutral
4,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),거품이나 향은 무난한데 새치 커버력이 홍보 대비 좀 아쉽네요,탈모,negative


In [38]:
grouped = df3.groupby(["shampoo"], observed = True)
gp_keys = list(grouped.groups.keys())
df3 = pd.DataFrame(columns=['shampoo', 'aspect', 'sentiment', 'pair_count', 'pair_ratio'])

for key in gp_keys:
    gp = grouped.get_group((key,))
    
    count_dict = count_pairs(gp)
    db_df = pd.Series(count_dict).reset_index()
    db_df.columns = ['aspect', 'sentiment', 'pair_count']
    db_df = db_df.sort_values(by="aspect")
    db_df["pair_count"] = db_df["pair_count"].astype(int)
    db_df = compute_ratio(db_df)
    db_df["shampoo"] = [key] * len(db_df)

    df3 = pd.concat([df3, db_df], axis = 0, ignore_index=True)
    df3["pair_count"] = df3["pair_count"].astype(int)

C:\Users\원유진\AppData\Local\Temp\ipykernel_26016\4067978117.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df3 = pd.concat([df3, db_df], axis = 0, ignore_index=True)


In [39]:
df3.head()

,shampoo,aspect,sentiment,pair_count,pair_ratio
0,TS BD샴푸 500g,가격,negative,15,0.6818
1,TS BD샴푸 500g,가격,neutral,0,0.0000
2,TS BD샴푸 500g,가격,positive,7,0.3182
3,TS BD샴푸 500g,거품,positive,57,0.8507
4,TS BD샴푸 500g,거품,neutral,1,0.0149


In [40]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11875 entries, 0 to 11874
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   shampoo     11875 non-null  object 
 1   aspect      11875 non-null  object 
 2   sentiment   11400 non-null  object 
 3   pair_count  11875 non-null  int32  
 4   pair_ratio  11875 non-null  float64
dtypes: float64(1), int32(1), object(3)
memory usage: 417.6+ KB


In [41]:
df3.shape

(11875, 5)

In [192]:
df3.to_csv('./db_aspectsentiment.csv', index=False)

--- 
# df4 - AspectOpinion 

In [48]:
df["labels"] = df["annotation"].map(extract_labels)

In [50]:
df4_data = []
for i in range(len(df)):
    review = df.loc[i, "리뷰"]
    shampoo = df.loc[i, "제품명"]
    writer = df.loc[i, "작성자"]
    for aspect, sentiment in df.loc[i, "labels"]:
        df4_data.append({
            "shampoo" : shampoo,
            "review" : review,
            "writer" : writer, 
            "aspect" : aspect,
            "sentiment" : sentiment
        })
df4 = pd.DataFrame(df4_data)

In [51]:
df4.head()

,shampoo,review,writer,aspect,sentiment
0,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),일주일째 쓰는 중인데 좋은 거 같기도 하고 아닌 것 같기도 하고 한 달 뒤 후기로..ㄹ,용자입니다,없음,None
1,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),거품이나 향은 무난한데 새치 커버력이 홍보 대비 좀 아쉽네요,레몽몽,세정,negative
2,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),거품이나 향은 무난한데 새치 커버력이 홍보 대비 좀 아쉽네요,레몽몽,거품,neutral
3,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),거품이나 향은 무난한데 새치 커버력이 홍보 대비 좀 아쉽네요,레몽몽,향,neutral
4,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),거품이나 향은 무난한데 새치 커버력이 홍보 대비 좀 아쉽네요,레몽몽,탈모,negative


In [52]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383197 entries, 0 to 383196
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   shampoo    383197 non-null  object
 1   review     383197 non-null  object
 2   writer     383197 non-null  object
 3   aspect     383197 non-null  object
 4   sentiment  360102 non-null  object
dtypes: object(5)
memory usage: 14.6+ MB


In [53]:
df4.shape

(383197, 5)

In [58]:
df4 = df4.drop_duplicates(subset = ["shampoo", "review", "writer", "aspect", "sentiment"])

In [59]:
df4.head(30)

,shampoo,review,writer,aspect,sentiment
0,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),일주일째 쓰는 중인데 좋은 거 같기도 하고 아닌 것 같기도 하고 한 달 뒤 후기로..ㄹ,용자입니다,없음,None
1,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),거품이나 향은 무난한데 새치 커버력이 홍보 대비 좀 아쉽네요,레몽몽,세정,negative
2,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),거품이나 향은 무난한데 새치 커버력이 홍보 대비 좀 아쉽네요,레몽몽,거품,neutral
3,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),거품이나 향은 무난한데 새치 커버력이 홍보 대비 좀 아쉽네요,레몽몽,향,neutral
4,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),거품이나 향은 무난한데 새치 커버력이 홍보 대비 좀 아쉽네요,레몽몽,탈모,negative
5,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),얼마 전에 구매했는데 쿠폰이 있어서 또 구매했어요 미리 쟁여 둘려 구요,k202401158****,없음,None
6,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),염색 하지 않고 몇 년째 계속 사용하고 있어요 남편 덕분에 알게 되어서 지금까지 같...,k202401158****,없음,None
7,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),남편 40대 되고 흰 머리가 많아 이걸 사줬는데 그레이 정도는 되는 것 같아요,sweetvio****,없음,None
8,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다는 사랑입니다. 친구들에게 추천하고 싶어요! 가격이 합리적이라 더 만족스러워요.,운영자,가격,positive
9,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),항성 사용하고 있어요굿 몇 년째 모다모다샴푸만 사용해 요항성 사용하고 있어요굿 몇 ...,puln****,자극,negative


In [205]:
df4.to_csv('./db_aspectopinion.csv', index=False)